In [39]:
#import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.models import Model#Sequential, Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
#from keras.optimizers import SGD
from keras.layers.convolutional import MaxPooling2D, Conv2D
#from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation, Conv2DTranspose, Input, concatenate, Cropping2D
from keras.layers.merge import concatenate
from keras import backend as K
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.models import Model, load_model


In [22]:
data = np.random.random((5,400,304,29))
#data = tf.convert_to_tensor(data,dtype=tf.float32)
linhas=data.shape[1]
colunas= data.shape[2]
canais = data.shape[3]

from ParamConfig import *
from PathConfig import *
from LibConfig import *


X_train,Y_train  = DataLoad_Train(train_size=TrainSize,train_data_dir=train_data_dir, \
                                                                 data_dim=DataDim,in_channels=Inchannels, \
                                                                 model_dim=ModelDim,data_dsp_blk=data_dsp_blk, \
                                                                 label_dsp_blk=label_dsp_blk,start=1, \
                                                                 datafilename=datafilename,dataname=dataname, \
                                                                 truthfilename=truthfilename,truthname=truthname, folder_dataset=folder_dataset)
# Change data type (numpy --> tensor)


Y_train = Y_train/1000

georec1
vmodel1
georec2
vmodel2
georec3
vmodel3
georec4
vmodel4


In [25]:
print (X_train.shape)
print (Y_train[1].shape)


(4, 400, 304, 29)
(202, 302, 1)


In [45]:
def apply_maxPooling(camada):
    camada = MaxPooling2D(pool_size=(2,2),padding='same') (camada)
    return camada

def reduzir(camada,out_size):
    camada = Conv2D(filters=out_size, kernel_size=(3, 3), input_shape=(linhas, colunas, canais), strides=1, padding='same') (camada)
    camada = BatchNormalization() (camada)
    camada = Activation('relu') (camada)
    
    #reaplicando novamente no dado
    camada = Conv2D(filters=out_size, kernel_size=(3, 3), input_shape=(linhas, colunas, canais), strides=1, padding='same') (camada)
    camada = BatchNormalization() (camada)
    camada = Activation('relu') (camada)
    
    return camada


def reduzir_last(camada,out_size):
    camada = Conv2D(filters=out_size, kernel_size=(3, 3), input_shape=(linhas, colunas, canais), strides=1, padding='same') (camada)
    camada = BatchNormalization() (camada)
    camada = Activation('relu') (camada)

    return camada

def aumentar_last(camada_corrente,camada_de_reducao_correspondente,out_size):
    camada = Conv2DTranspose(filters=out_size,kernel_size=(2,2),strides=2,padding='same') (camada_corrente)
    camada = concatenate([camada,camada_de_reducao_correspondente],axis=3) #aqui concateno o dado após a deconvolução com o dado correspondente na camada de redução
    camada = reduzir_last(camada,out_size)

    return camada

#data = np.random.random((500,400,304,29))
#data = tf.convert_to_tensor(data,dtype=tf.float32)
#linhas=data.shape[1]
#colunas= data.shape[2]
#canais = data.shape[3]


def aumentar(camada_corrente,camada_de_reducao_correspondente,out_size):
    camada = Conv2DTranspose(filters=out_size,kernel_size=(2,2),strides=2,padding='same') (camada_corrente)
    camada = concatenate([camada,camada_de_reducao_correspondente],axis=3) #aqui concateno o dado após a deconvolução com o dado correspondente na camada de redução
    camada = reduzir(camada,out_size)

    return camada


def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)



def create_model(linhas,colunas,canais):
    inputs = Input((linhas,colunas,canais))
    
    filtros=[64,128,256,512,1024]
    c1 = reduzir(inputs,filtros[0])
    c2 = reduzir(apply_maxPooling(c1),filtros[1])
    c3 = reduzir(apply_maxPooling(c2),filtros[2])
    c4 = reduzir(apply_maxPooling(c3),filtros[3])
    c5 = reduzir(apply_maxPooling(c4),filtros[4])
    

    c6 = aumentar(c5,c4,filtros[3])
    c7 = aumentar(c6,c3,filtros[2])
    c8 = aumentar(c7,c2,filtros[1])
    c9 = aumentar_last(c8,c1,filtros[0])
    
    c9 = Cropping2D(cropping=(99, 1)) (c9) #fiz um crop na imagem que era (400,304) e deixei com (202,302)
    c9 = reduzir_last(c9,filtros[0])
    c9 = Conv2D(filters=1, kernel_size=(1, 1), strides=1) (c9)

    model = Model(inputs=[inputs],outputs=[c9])
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    model.summary()
    return model

model = create_model(linhas,colunas,canais)

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 400, 304, 29) 0                                            
__________________________________________________________________________________________________
conv2d_96 (Conv2D)              (None, 400, 304, 64) 16768       input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_91 (BatchNo (None, 400, 304, 64) 256         conv2d_96[0][0]                  
__________________________________________________________________________________________________
activation_91 (Activation)      (None, 400, 304, 64) 0           batch_normalization_91[0][0]     
____________________________________________________________________________________________

In [46]:
# Fit model
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('model-dsbowl2018-1.h5', verbose=1, save_best_only=True)
results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=2, epochs=10, 
                    callbacks=[earlystopper, checkpointer])

Train on 3 samples, validate on 1 samples
Epoch 1/10
3/3 [==============================] - 2s 750ms/step - loss: 5.5996 - mse: 5.5996 - val_loss: 5.9399 - val_mse: 5.9399

Epoch 00001: val_loss improved from inf to 5.93994, saving model to model-dsbowl2018-1.h5
Epoch 2/10
3/3 [==============================] - 1s 175ms/step - loss: 3.4412 - mse: 3.4412 - val_loss: 6.4945 - val_mse: 6.4945

Epoch 00002: val_loss did not improve from 5.93994
Epoch 3/10
3/3 [==============================] - 1s 170ms/step - loss: 2.0134 - mse: 2.0134 - val_loss: 24.2551 - val_mse: 24.2551

Epoch 00003: val_loss did not improve from 5.93994
Epoch 4/10
3/3 [==============================] - 1s 171ms/step - loss: 1.3540 - mse: 1.3540 - val_loss: 189.5072 - val_mse: 189.5072

Epoch 00004: val_loss did not improve from 5.93994
Epoch 5/10
3/3 [==============================] - 1s 171ms/step - loss: 0.9248 - mse: 0.9248 - val_loss: 516.2706 - val_mse: 516.2706

Epoch 00005: val_loss did not improve from 5.93994

In [43]:
# Predict on train, val and test
model = load_model('model-dsbowl2018-1.h5')
preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# Create list of upsampled test masks
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))

1/1 [==============================] - 0s 67ms/step


NameError: name 'X_test' is not defined